In [1]:
import torch
from torch_geometric.data import InMemoryDataset, download_url, Data, Batch
from torch import nn
from torch.nn import functional as F
import os
import pandas as pd
import numpy as np
import pickle
import itertools
import jax
from jax import numpy as jnp
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import MinMaxScaler
import mendeleev
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler, HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
import os

In [2]:
import os
from torch.utils.data import TensorDataset, DataLoader

In [3]:
files_train = os.listdir("/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/train/")

In [4]:
data = []
labels = []
directory = "/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/train/"
files_train = os.listdir(directory)
for file in np.sort(files_train):
    path = os.path.join(directory, file)
    if file[0] == "d":
        data.append(pd.read_hdf(path))
    else:
        labels.append(pd.read_hdf(path))

In [5]:
y_train = pd.concat(labels)
X_train = pd.concat(data)

In [6]:
data = []
labels = []
directory = "/mnt/10edb508-27ad-4f92-9467-37a536784b53/temp/test/"
files_test = os.listdir(directory)
for file in np.sort(files_test):
    path = os.path.join(directory, file)
    if file[0] == "d":
        data.append(pd.read_hdf(path))
    else:
        labels.append(pd.read_hdf(path))

In [7]:
X_temp_train = X_train[(y_train[0] == 1).to_numpy()]
y_temp_train = y_train[(y_train[0] == 1).to_numpy()]

In [8]:
y_test = pd.concat(labels)
X_test = pd.concat(data)

In [9]:
X_temp_test = X_test[(y_test[0] == 1).to_numpy()]
y_temp_test = y_test[(y_test[0] == 1).to_numpy()]

In [10]:
train_dataset = TensorDataset(torch.Tensor(X_temp_train.to_numpy()),torch.Tensor(y_temp_train.to_numpy())) # create your datset
train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=4096, shuffle=True, drop_last=True)
test_dataset = TensorDataset(torch.Tensor(X_temp_test.to_numpy()),torch.Tensor(y_temp_test.to_numpy())) # create your datset
test_dataloader = DataLoader(test_dataset, num_workers=12, batch_size=4096, shuffle=True)

In [11]:
del X_train, X_test, data, labels

In [12]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error

In [13]:
# reg = RandomForestRegressor(n_estimators=8, max_depth = 16)

In [14]:
# reg.fit(X_temp_train,y_temp_train)
# y_pred = reg.predict(X_temp_test)
# print(mean_absolute_error(y_pred, y_temp_test))

In [15]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class ResNetGated(nn.Module):
    def __init__(self, init_dim, hidden_dim, layers, p_dropout):
        super().__init__()
        self.p_dropout = p_dropout
        assert layers > 0
        self.layers = nn.ModuleList([nn.Sequential(nn.Linear(init_dim, hidden_dim),
                             nn.ReLU(),
                             nn.BatchNorm1d(hidden_dim),
                             nn.Linear( hidden_dim, init_dim)) for i in range(layers)])
        self.gates = nn.Parameter(torch.Tensor(layers))
        self.layers.apply(init_weights)
    def forward(self, x):
        range_gates = torch.sigmoid(self.gates)
        for i, layer in enumerate(self.layers):
            x = F.relu(x)
            x = (range_gates[i])*layer(x) + (1-range_gates[i])*x
        return x

    
class Net(torch.nn.Module):
    def __init__(self, init_dim, hidden_dim, hidden_width, layers, p_dropout):
        super().__init__()
        self.dr = nn.Sequential(nn.Linear(init_dim, 512),
                               nn.ReLU(),
                               nn.Dropout(p=p_dropout),
                               nn.Linear(512, hidden_dim))
        # self.ad = nn.Linear(init_dim, hidden_dim)
        self.fc = ResNetGated(hidden_dim, hidden_width, layers, p_dropout)
        self.reg = nn.Linear(hidden_dim, 1)
        self.fc.apply(init_weights)
        self.reg.apply(init_weights)
        self.dr.apply(init_weights)
    def forward(self, x):
        x = self.dr(x)
        x = self.fc(x)
        x = self.reg(x)
        return x

In [16]:
def train(model, device, data ,loss_fn, optimizer):
    model.train()
    train_losses = []
    optimizer.zero_grad()
    for i, batch in enumerate(data):
        data, labels = batch[0].to(device), batch[1][:, 1].to(device)
        logits = model(data)
        loss = loss_fn()
        output=loss(logits.squeeze(), labels.squeeze())
        output.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
        optimizer.step()
        train_loss = output.data.cpu().numpy()
        train_losses.append(train_loss)    
    return np.mean(train_losses)

### Testing function
def test(model, device, data, loss_fn):
    # Set evaluation mode for encoder and decoder
    model.eval()
    test_losses = []
    with torch.no_grad(): # No need to track the gradients
        for i, batch in enumerate(data):
            data, labels = batch[0].to(device), batch[1][:, 1].to(device)
            logits = model(data)
            loss = loss_fn()
            output=loss(logits.squeeze(), labels.squeeze())
            test_loss = output.data.cpu().numpy()
            test_losses.append(test_loss)  
    return np.mean(test_losses)

In [17]:
### Define the loss function
loss_fn = nn.MSELoss

lr= 0.0001
weight_decay = 0.00001
hidden_dim = 256
hidden_width=4096
layers=2
p_dropout = 0.05
### Set the random seed for reproducible results
torch.manual_seed(0)

model = Net(776, hidden_dim, hidden_width, layers, p_dropout=p_dropout)
params_to_optimize = [
    {'params': model.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=weight_decay)
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
model.to(device)

Selected device: cuda


Net(
  (dr): Sequential(
    (0): Linear(in_features=776, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.05, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
  )
  (fc): ResNetGated(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=256, out_features=4096, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Linear(in_features=4096, out_features=256, bias=True)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=4096, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Linear(in_features=4096, out_features=256, bias=True)
      )
    )
  )
  (reg): Linear(in_features=256, out_features=1, bias=True)
)

In [18]:
import datetime
date = datetime.datetime.today().strftime('%Y-%m-%d-%H:%M:%S')

RUN = date

In [19]:
best_loss = 1000
num_epochs = 100
diz_loss = {'train_loss':[],'val_loss':[]}
decay = 0.98
for epoch in range(num_epochs):
    if epoch == 0:
        for par in optim.param_groups:
            par["lr"] = 0.0001
    elif epoch< 10:
        for par in optim.param_groups:
            par["lr"] = par["lr"] * 1.2
    else:
        for par in optim.param_groups:
            par["lr"] = par["lr"] * decay
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/100 	 train loss 17.21969985961914 	 	 val loss 0.8408823013305664

 EPOCH 2/100 	 train loss 1.0096327066421509 	 	 val loss 0.9979063272476196

 EPOCH 3/100 	 train loss 0.7975667715072632 	 	 val loss 0.7027420401573181

 EPOCH 4/100 	 train loss 0.7499180436134338 	 	 val loss 1.0264877080917358

 EPOCH 5/100 	 train loss 0.7240357995033264 	 	 val loss 0.529172420501709

 EPOCH 6/100 	 train loss 0.7814425826072693 	 	 val loss 0.6588134765625

 EPOCH 7/100 	 train loss 0.6771257519721985 	 	 val loss 0.9054681062698364

 EPOCH 8/100 	 train loss 0.6768056154251099 	 	 val loss 0.69672030210495

 EPOCH 9/100 	 train loss 0.6296425461769104 	 	 val loss 0.7710046172142029

 EPOCH 10/100 	 train loss 0.661254346370697 	 	 val loss 0.776839554309845

 EPOCH 11/100 	 train loss 0.5719884634017944 	 	 val loss 0.963376522064209

 EPOCH 12/100 	 train loss 0.57888263463974 	 	 val loss 0.39220279455184937

 EPOCH 13/100 	 train loss 0.4494883418083191 	 	 val loss 0.4817401468

In [20]:
train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=9192, shuffle=True, drop_last=True)

In [21]:
diz_loss = {'train_loss':[],'val_loss':[]}
decay = 0.98
num_epochs = 200
size = 4096
for epoch in range(num_epochs):
    if epoch%50 == 0:
        for par in optim.param_groups:
            par["weight_decay"] = par["weight_decay"]/2
            size = size * 2
            train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=size, shuffle=True, drop_last=True)
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/200 	 train loss 0.06019546836614609 	 	 val loss 0.08806730061769485

 EPOCH 2/200 	 train loss 0.06040557473897934 	 	 val loss 0.08503196388483047

 EPOCH 3/200 	 train loss 0.057593461126089096 	 	 val loss 0.09711442142724991

 EPOCH 4/200 	 train loss 0.06583303213119507 	 	 val loss 0.10313760489225388

 EPOCH 5/200 	 train loss 0.05989842116832733 	 	 val loss 0.08153393119573593

 EPOCH 6/200 	 train loss 0.057396434247493744 	 	 val loss 0.08372542262077332

 EPOCH 7/200 	 train loss 0.06888747960329056 	 	 val loss 0.09384352713823318

 EPOCH 8/200 	 train loss 0.0629672184586525 	 	 val loss 0.08295179903507233

 EPOCH 9/200 	 train loss 0.06539981067180634 	 	 val loss 0.11212271451950073

 EPOCH 10/200 	 train loss 0.08305806666612625 	 	 val loss 0.1172107383608818

 EPOCH 11/200 	 train loss 0.06543964147567749 	 	 val loss 0.091547392308712

 EPOCH 12/200 	 train loss 0.057070955634117126 	 	 val loss 0.08301211148500443

 EPOCH 13/200 	 train loss 0.05961901

In [22]:
decay = 0.98
num_epochs = 500
size = 4096
for epoch in range(num_epochs):
    # if epoch%50 == 0:
    #     for par in optim.param_groups:
    #         par["weight_decay"] = par["weight_decay"]/2
    #         size = size * 2
    #         train_dataloader = DataLoader(train_dataset, num_workers=12, batch_size=size, shuffle=True, drop_last=True)
    train_loss = train(model, device, train_dataloader, loss_fn, optim)
    test_loss = test(model, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), "./saved_models/type0_{}.pth".format(RUN))


 EPOCH 1/500 	 train loss 0.032631766051054 	 	 val loss 0.07134019583463669

 EPOCH 2/500 	 train loss 0.03316415473818779 	 	 val loss 0.07160177081823349

 EPOCH 3/500 	 train loss 0.03387696295976639 	 	 val loss 0.07211233675479889

 EPOCH 4/500 	 train loss 0.034450456500053406 	 	 val loss 0.07589326798915863

 EPOCH 5/500 	 train loss 0.033498525619506836 	 	 val loss 0.07616861164569855

 EPOCH 6/500 	 train loss 0.032203011214733124 	 	 val loss 0.07563415169715881

 EPOCH 7/500 	 train loss 0.03481365367770195 	 	 val loss 0.07256605476140976

 EPOCH 8/500 	 train loss 0.03414592891931534 	 	 val loss 0.07382109016180038

 EPOCH 9/500 	 train loss 0.032610539346933365 	 	 val loss 0.07311073690652847

 EPOCH 10/500 	 train loss 0.03571043163537979 	 	 val loss 0.07205426692962646

 EPOCH 11/500 	 train loss 0.035103894770145416 	 	 val loss 0.07202353328466415

 EPOCH 12/500 	 train loss 0.03690194711089134 	 	 val loss 0.07252496480941772

 EPOCH 13/500 	 train loss 0.0365